In [5]:
import pandas as pd

# Load the Excel file
file_path = r"C:\Users\CHAKU FOODS\Documents\Inputs Matching.xlsx"  # Replace with the correct file path
excel_data = pd.ExcelFile(file_path)

# Load the sheets
group_crop_mapping_inputs = excel_data.parse('group_crop_mapping_inputs')
new_inputs_table = excel_data.parse('New Inputs Table')

# Identify input columns, excluding "Inputs used/Other (manually specify)"
input_columns = [col for col in group_crop_mapping_inputs.columns if "Inputs used" in col and "Other (manually specify)" not in col]

# Extract existing entries from the "New Inputs Table" to avoid duplicates
existing_entries = set(zip(new_inputs_table['last_name'], new_inputs_table['first_name'], new_inputs_table['input_type']))

# Prepare a list to collect the new data for the "New Inputs Table"
new_entries = []

# Iterate through each row in the input columns
for _, row in group_crop_mapping_inputs.iterrows():
    for input_col in input_columns:
        if row[input_col] == 1:  # Check if the value is 1
            input_type = input_col.split('/')[-1]  # Extract the input type from the column name
            
            # Check if the combination already exists
            entry_key = (row['last_name'], row['first_name'], input_type)
            if entry_key not in existing_entries:
                new_entries.append({
                    'last_name': row['last_name'],
                    'first_name': row['first_name'],
                    'input_type': input_type,
                    'details': None  # Placeholder for 'details' column
                })
                existing_entries.add(entry_key)  # Add to the existing entries to prevent duplicates

# Convert the new entries to a DataFrame
new_entries_df = pd.DataFrame(new_entries)

# Append to the existing "New Inputs Table"
updated_new_inputs_table = pd.concat([new_inputs_table, new_entries_df], ignore_index=True)

# Save the updated table back to a new Excel file
output_path = 'Updated_Inputs_Matching.xlsx'  # Replace with your desired output file path

# Write the sheets to the new Excel file using the default engine
with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
    group_crop_mapping_inputs.to_excel(writer, sheet_name='group_crop_mapping_inputs', index=False)
    updated_new_inputs_table.to_excel(writer, sheet_name='New Inputs Table', index=False)

print(f"Updated file saved to {output_path}")


Updated file saved to Updated_Inputs_Matching.xlsx
